# GOSTrocks Primer

GOSTrocks is an internal sandbox repository that contains commonly used geospatial functions and operations. This notebooks demonstrates the usage of selected functions.

The repository also contains customized code to process specific datasets (GHSL, OpenStreetMap, Nighttime Lights, Fathom). This code is in constant development as project requirements evolve over time.

## Installation

### From PyPI

```
pip install GOSTrocks
```

### From Source

```
git clone https://github.com/worldbank/GOSTrocks.git
cd GOSTrocks
pip install .
```

## Import Functions

GOSTrocks is broken down into modules of functions that share common elements.

In [ ]:
import GOSTrocks.dataMisc as dMisc  # search and download specific datasets
import GOSTrocks.rasterMisc as rMisc  # Raster tools!

## Raster Example

One of the most recurring workflows is to clip and combine raster data from different sources.

Here, we work with population data from WorldPop, and a friction surface from the Malaria Atlas Project, using Ghana as our area of interest.

They are both global datasets with ~1km resolution, so they both need to be clipped and standardized for analysis.

We will use administrative boundaries from the GeoBoundaries API. 

In [ ]:
import os
from os.path import join, exists
import urllib.request

import rasterio as rio
from rasterio.plot import show

import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [ ]:
country = "Ghana"
iso = "GHA"

### Get Boundaries

In [ ]:
dMisc.get_geoboundaries?

In [ ]:
gdf = dMisc.get_geoboundaries(iso, level="ADM0")
gdf.plot()

### Download Population Data

In [ ]:
wp_url = f"https://data.worldpop.org/GIS/Population/Global_2000_2020_1km/2020/{iso.upper()}/{iso.lower()}_ppp_2020_1km_Aggregated.tif"
wp_url

In [ ]:
wp_path = join(
    "..", "data", f"{iso}_ppp_2020_1km_Aggregated_UNadj.tif"
)  # Download from link above
if not exists(wp_path):
    urllib.request.urlretrieve(wp_url, wp_path)

In [ ]:
src_pop = rio.open(wp_path)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
show(src_pop, norm=colors.PowerNorm(gamma=0.5), cmap="viridis", ax=ax)
plt.axis("off")

### Clip Global Friction Surface

In [ ]:
gfs_path = "J:/Data/GLOBAL/INFRA/FRICTION_2020/2020_motorized_friction_surface.geotiff"
src_travel = rio.open(gfs_path)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
show(src_travel, norm=colors.PowerNorm(gamma=0.2), cmap="magma", ax=ax)
plt.axis("off")

In [ ]:
rMisc.clipRaster?

In [ ]:
out_travel = join("..", "data", f"{iso.upper()}_2020_motorized_friction_surface.tif")
surface, surface_meta = rMisc.clipRaster(
    src_travel, gdf, outFile=out_travel, crop=False
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
show(surface, norm=colors.PowerNorm(gamma=0.2), cmap="magma", ax=ax)
plt.axis("off")

### Standardize Rasters

Align the population raster to the friction surface, ensuring that they have the same extent and resolution.

In [ ]:
rMisc.standardizeInputRasters?

In [ ]:
src_travel_clipped = rio.open(out_travel)

In [ ]:
pop_std, pop_meta = rMisc.standardizeInputRasters(
    src_pop, src_travel_clipped, resampling_type="nearest"
)

Original population raster metadata:

In [ ]:
src_pop.meta

New population metadata aligned with travel surface:

In [ ]:
pop_meta, surface_meta

In [ ]:
src_pop.close()
src_travel.close()
src_travel_clipped.close()

In [ ]:
os.remove(out_travel)
os.remove(wp_path)